In [74]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from math import sqrt

In [75]:
data = pd.read_csv("climate.csv")
data.drop_duplicates()
data["date"] = data["date"].apply(pd.to_datetime)
data["date"] = (data["date"]).astype(np.int64) / 1000000000
data.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,1.356998e+09,10.000000,84.500000,0.000000,1015.666667
1,1.357085e+09,7.400000,92.000000,2.980000,1017.800000
2,1.357171e+09,7.166667,87.000000,4.633333,1018.666667
3,1.357258e+09,8.666667,71.333333,1.233333,1017.166667
4,1.357344e+09,6.000000,86.833333,3.700000,1016.500000


In [76]:
start_ind = 300
end_ind = 450
n = round(0.8 * (end_ind - start_ind))

In [77]:
x = data["date"][start_ind:end_ind].to_numpy() - data["date"][start_ind]
y = data["meantemp"][start_ind:end_ind].to_numpy()

In [78]:
train_x = x[0:n]
train_y = y[0:n]

test_x = x[n:]
test_y = y[n:]

In [79]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_x, y=train_y, name="train"))
fig.add_trace(go.Scatter(x=test_x, y=test_y, name="test"))
fig.update_layout(title="Выборки",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

МНК

In [80]:
from scipy.optimize import curve_fit

In [81]:
def func(x, c0, c1, c2):
  return c0 + c1 * x + c2 * x ** 2

In [82]:
popt, pcov = curve_fit(func, train_x, train_y)
c0, c1, c2 = popt[0], popt[1], popt[2]

res_y = func(train_x, c0, c1, c2)

In [83]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_x, y=train_y, name="Исходные"))
fig.add_trace(go.Scatter(x=train_x, y=res_y, name="Результат"))
fig.update_layout(title="МНК (train)",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

In [84]:
print("МНК train rmse:", sqrt(mean_squared_error(train_y, res_y)))

МНК train rmse: 1.709096280321927


In [85]:
res_y = func(test_x, c0, c1, c2)

In [86]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_x, y=test_y, name="Исходные"))
fig.add_trace(go.Scatter(x=test_x, y=res_y, name="Результат"))
fig.update_layout(title="МНК (test)",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

In [87]:
print("МНК test rmse:", sqrt(mean_squared_error(test_y, res_y)))

МНК test rmse: 2.516422318443904


KNN

In [88]:
from sklearn.neighbors import KNeighborsRegressor

In [89]:
model = KNeighborsRegressor(n_neighbors=3)

model.fit(train_x.reshape(-1, 1), train_y)
res_y = model.predict(train_x.reshape(-1, 1))

In [90]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_x, y=train_y, name="Исходные"))
fig.add_trace(go.Scatter(x=train_x, y=res_y, name="Результат"))
fig.update_layout(title="KNN (train)",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

In [91]:
print("KNN train rmse:", sqrt(mean_squared_error(train_y, res_y)))

KNN train rmse: 0.6671981634325846


In [92]:
res_y = model.predict(test_x.reshape(-1, 1))

In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_x, y=test_y, name="Исходные"))
fig.add_trace(go.Scatter(x=test_x, y=res_y, name="Результат"))
fig.update_layout(title="KNN (test)",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

In [94]:
print("KNN test rmse:", sqrt(mean_squared_error(test_y, res_y)))

KNN test rmse: 4.977066879989775


ARIMA

In [95]:
from statsmodels.tsa.arima.model import ARIMA

In [96]:
history = [y for y in train_y]
predictions = list()

for i in range(len(test_y)):
  model = ARIMA(history, order=(1, 1, 0))
  model_fit = model.fit()
  yhat = model_fit.forecast()[0]
  predictions.append(yhat)
  obs = test_y[i]
  history.append(obs)

In [97]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_x, y=train_y, name="train"))
fig.add_trace(go.Scatter(x=test_x, y=test_y, name="test"))
fig.add_trace(go.Scatter(x=test_x, y=predictions, name="ARIMA результат"))
fig.update_layout(title="ARIMA",
                  xaxis_title="Дата",
                  yaxis_title="Температура")

In [98]:
print("ARIMA test rmse:", sqrt(mean_squared_error(test_y, predictions)))

ARIMA test rmse: 1.6063632657333258
